## READ ME FIRST

This notebook produces a weekly release from the `develop` branch of Cytoscape.

The weekly release consists of Install4j generated installers that can be put on a public server to be downloaded by testers. This script will NOT push any code changes or resources to GitHub or Nexus or update any additional resources like the manual or API docs. 

This notebook is configured for the build server on which it is run. Notes have been made where local environment variables have been set.

Before starting this process, it is a good idea to Restart and Clear Output for this notebook's Kernel.


### 1. Set up Build Environment

This sets up the build environment, including Java and Maven versions, and the root directory of the build.


In [1]:
from subprocess import Popen, PIPE
import os
import shutil

Set the notebook directory. Note that this is system dependent.

In [2]:
NOTEBOOK_DIR = '/home/cybuilder/cytoscape-admin-scripts'

print(NOTEBOOK_DIR)

/home/cybuilder/cytoscape-admin-scripts


Set the Java environment. This is Java 11 for Cytoscape 3.8.0 and above.

Note that this is system dependent, and may need to be changed to reflect the system's configuration.

In [3]:
%env JAVA_HOME=/usr/lib/jvm/jre-11-openjdk

env: JAVA_HOME=/usr/lib/jvm/jre-11-openjdk


Set the MAVEN_HOME environment variable, as well as point the path to the correct Maven binaries. 

Note that this is system dependent and may need to be changed to reflect the system's configuration.

In [4]:
%env MAVEN_HOME=/opt/maven
%env PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin

env: MAVEN_HOME=/opt/maven
env: PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin


Prepare the target directory and clone the cytoscape git repo.

In [5]:
print('Changing to directory: ' + NOTEBOOK_DIR)
os.chdir(NOTEBOOK_DIR)

# Point to build location (the directory to clone parent cytoscape into)
BUILD_PARENT_DIR = os.path.join(os.getcwd(), 'install4j8_build')
if not os.path.exists(BUILD_PARENT_DIR):
    os.mkdir(BUILD_PARENT_DIR)
else:
    shutil.rmtree(BUILD_PARENT_DIR)
    os.mkdir(BUILD_PARENT_DIR)

os.chdir(BUILD_PARENT_DIR)
![[ -d cytoscape ]] || git clone https://github.com/cytoscape/cytoscape
CYTOSCAPE_ROOT_DIR = os.path.join(BUILD_PARENT_DIR, 'cytoscape')
CYTOSCAPE_DIR = os.path.join(CYTOSCAPE_ROOT_DIR, 'cytoscape')

def cd(directory=BUILD_PARENT_DIR, *subdirs):
    if subdirs:
        directory = os.path.join(directory, *subdirs)
    if os.getcwd() != directory:
        os.chdir(directory)

Changing to directory: /home/cybuilder/cytoscape-admin-scripts
Cloning into 'cytoscape'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 541 (delta 3), reused 4 (delta 1), pack-reused 529
Receiving objects: 100% (541/541), 363.77 KiB | 0 bytes/s, done.
Resolving deltas: 100% (256/256), done.


In [6]:
STARTING_BRANCH = 'develop'     # develop for major release, release/3.X.X for minor release

## 2. Pull the develop branch of Cytoscape

Note that to execute the checkout step here, you will need to have set up an SSH key on this machine that does not use password validation, or else parts of the build will be stall when they require input. 

In [7]:
cd(CYTOSCAPE_ROOT_DIR)
![[ -d cytoscape ]] || ./cy.sh init
cd(CYTOSCAPE_DIR)
!./cy.sh run-all "git checkout {STARTING_BRANCH}"

Target directory = 
Cytoscape project will be cloned to: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape
Cloning into 'cytoscape'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 541 (delta 3), reused 4 (delta 1), pack-reused 529
Receiving objects: 100% (541/541), 363.77 KiB | 0 bytes/s, done.
Resolving deltas: 100% (256/256), done.
Cloning: parent (URI = git@github.com:cytoscape/cytoscape-parent.git)
Cloning into 'parent'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1610 (delta 20), reused 17 (delta 1), pack-reused 1569
Receiving objects: 100% (1610/1610), 253.19 KiB | 0 bytes/s, done.
Resolving deltas: 100% (605/605), done.
~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/parent ~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape
Alread

## 2a. Reset

In [8]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git clean -f -d'
!./cy.sh run-all 'git reset --hard'

------------------------------------------------------------------------
Executing command: git clean -f -d
--in .
------------------------------------------------------------------------
--in parent
------------------------------------------------------------------------
--in api
------------------------------------------------------------------------
--in impl
------------------------------------------------------------------------
--in support
------------------------------------------------------------------------
--in gui-distribution
------------------------------------------------------------------------
--in app-developer
------------------------------------------------------------------------
------------------------------------------------------------------------
Executing command: git reset --hard
--in .
HEAD is now at 87a9a80 Update README.md
------------------------------------------------------------------------
--in parent
HEAD is now at 5e23447 Add jacoco code coverage


## 2. Verify status

In [9]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git status'

------------------------------------------------------------------------
Executing command: git status
--in .
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in parent
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in api
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in impl
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in support
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in gui-distribution
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in app

## 3. Build Cytoscape and ensure no errors

This may take a while. Expect to build subrepos first before building from the root directory

Currently, you should expect two or so non-fatal javadoc-bundle-options related errors.

In [10]:
cd(CYTOSCAPE_DIR)
with open('build_output.txt', 'w') as outf:
    process = Popen('mvn clean install -Dmaven.test.skip=true'.split(' '), 
                stdout=outf,
                cwd=CYTOSCAPE_DIR)
    process.wait()

print("Showing ERROR lines in build...")
!cat build_output.txt | grep ERROR

Showing ERROR lines in build...
[ERROR] Error fetching link: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/api/app-api/target/javadoc-bundle-options. Ignored it.
[ERROR] Error fetching link: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/api/swing-app-api/target/javadoc-bundle-options. Ignored it.


## 4. Build Cytoscape installers

This requires Install4J to be configured on your machine and for the Install4j project file to point to the correct Mac signing file. The keystore password is set below (blank in this config), to be interpreted by the Install4j Maven plugin.

In [13]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
%env MAC_KEYSTORE_PASSWORD=
!mvn clean install -U

env: MAC_KEYSTORE_PASSWORD=
[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for org.cytoscape.distribution:packaging:jar:3.8.0-SNAPSHOT
[WARNING] 'build.plugins.plugin.version' for org.sonatype.install4j:install4j-maven-plugin is missing. @ org.cytoscape.distribution:packaging:[unknown-version], /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/pom.xml, line 104, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ----------------< org.cytoscape.distribution:packaging >----------------
[INFO] Building Cytoscape Release Packaging 3.8.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
Downloaded from central:

[INFO] No sources to compile
[INFO] 
[INFO] --- maven-resources-plugin:2.6:testResources (default-testResources) @ packaging ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/src/test/resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.0:testCompile (default-testCompile) @ packaging ---
[INFO] No sources to compile
[INFO] 
[INFO] --- maven-surefire-plugin:3.0.0-M3:test (default-test) @ packaging ---
[INFO] No tests to run.
[INFO] 
[INFO] --- jacoco-maven-plugin:0.8.4:report (report) @ packaging ---
[INFO] Skipping JaCoCo execution due to missing execution data file.
[INFO] 
[INFO] --- maven-jar-plugin:2.4:jar (default-jar) @ packaging ---
[INFO] Building jar: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/packaging-3.8.0-SNAPSHOT.jar
[INFO] 
[INFO] >>> maven-sourc

[INFO] 
[INFO] --- maven-install-plugin:2.4:install (default-install) @ packaging ---
[INFO] Installing /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/packaging-3.8.0-SNAPSHOT.jar to /home/cybuilder/.m2/repository/org/cytoscape/distribution/packaging/3.8.0-SNAPSHOT/packaging-3.8.0-SNAPSHOT.jar
[INFO] Installing /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/pom.xml to /home/cybuilder/.m2/repository/org/cytoscape/distribution/packaging/3.8.0-SNAPSHOT/packaging-3.8.0-SNAPSHOT.pom
[INFO] Installing /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/packaging-3.8.0-SNAPSHOT-sources.jar to /home/cybuilder/.m2/repository/org/cytoscape/distribution/packaging/3.8.0-SNAPSHOT/packaging-3.8.0-SNAPSHOT-sources.jar
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO]

## 5. Copying Cytoscape installers to weekly download page

When completed the builds can be found in `cytoscape/cytoscape/gui-distribution/packaging/target/media`

which should be copied to `/var/www/html/cytoscape-builds/Cytoscape-3.8/<DATE>`

